Import and install some necessary libraries

In [ ]:
import csv
import os
import re

In [ ]:
!pip install beautifulsoup4 --quiet
!pip install fake-useragent --quiet
!pip install selenium --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.8 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

Old function for scrapping data from the Indomaret website. No longer used because it is irrelevant, and there are frequent data misses.

In [ ]:
# old scrapper
def scrapper(key: str, categories: str, max_page: int, label: str):
    print(f"memulai scrapping untuk label: {label}\n")
    template_url: str = "https://www.klikindomaret.com/search/?key={key}&categories={categories}&productbrandid=&sortcol=&pagesize=54&page={page}&startprice=&endprice=&attributes=&ShowItem="

    products = []

    for page_index in range(1, max_page + 1):
        ua = UserAgent()
        options = Options()
        options.add_argument("-headless")
        options.add_argument(f"user-agent={ua.random}")
        driver: WebDriver = webdriver.Chrome(options=options)

        url: str = template_url.format(
            key=key,
            categories=categories,
            page=page_index,
        )
        print(f"percobaan ke-{page_index}: {url}")

        try:
            driver.get(url)

            try:
                wait = WebDriverWait(
                    driver=driver,
                    timeout=5,
                )
                wait.until(expected_conditions.presence_of_element_located((By.ID, 'site-content')))
            except:
                raise LookupError("tidak ada element dengan id site-content")

            content = driver.page_source
            soup = BeautifulSoup(content, features='html.parser')

            product_collection = soup.find('div', class_='product-collection')

            for item in product_collection.find_all('div', class_='item'):
                each_item_element = item.find('div', class_='each-item')

                wrp_media_element = each_item_element.find('div', class_='wrp-media')
                try:
                    image_url = wrp_media_element.find('img')['data-src']
                except:
                    image_url = ""

                wrp_content_element = each_item_element.find('div', class_='wrp-content')

                title = wrp_content_element.find('div', class_='title').text

                wrp_price_element = wrp_content_element.find('div', class_='wrp-price')

                # cek apakah ada discount atau tidak
                try:
                    if wrp_price_element.find('span', class_='strikeout disc-price'):
                        item_price = wrp_price_element.find('span', class_='discount').text
                        item_price = re.sub(r"<span>.*?</span>", "", item_price)
                    else:
                        price_element = wrp_price_element.find('div', class_='price')
                        item_price = price_element.find('span', class_='normal price-value').text
                except:
                    item_price = 'tidak ada'
                    print(f'gagal mendapatkan harga untuk item: {title}')

                products.append({
                    'title': title.strip(),
                    'normal_price': item_price.strip(),
                    'label': label,
                    'image_url': image_url,
                })

            print(f"berhasil melakukan percobaan ke-{page_index}")
        except Exception as e:
            print("terjadi error: ", e)

        driver.quit()

    return products

New function to scrapping Indomaret website based on category. In this function, some information is retrieved, such as product name, product price, and product image.

In [ ]:
# new scrapper by specific url
def scrapper_v2(
    category1: str,
    category2: str,
    max_page: int,
    label: str,
):
  print(f"memulai scrapping untuk label: {label}\n")
  template_url = 'https://www.klikindomaret.com/category/{category1}?categories={category2}&pagesize=54&page={page}'

  products = []

  for page_index in range(1, max_page + 1):
      ua = UserAgent()
      options = Options()
      options.add_argument("-headless")
      options.add_argument(f"user-agent={ua.random}")
      driver: WebDriver = webdriver.Chrome(options=options)

      url: str = template_url.format(
          category1=category1,
          category2=category2,
          page=page_index,
      )
      print(f"percobaan ke-{page_index}: {url}")

      try:
          driver.get(url)

          try:
              wait = WebDriverWait(
                  driver=driver,
                  timeout=5,
              )
              wait.until(expected_conditions.presence_of_element_located((By.ID, 'site-content')))
          except:
              raise LookupError("tidak ada element dengan id site-content")

          content = driver.page_source
          soup = BeautifulSoup(content, features='html.parser')

          product_collection = soup.find('div', class_='product-collection')

          for item in product_collection.find_all('div', class_='item'):
              each_item_element = item.find('div', class_='each-item')

              wrp_media_element = each_item_element.find('div', class_='wrp-media')
              try:
                  image_url = wrp_media_element.find('img')['data-src']
              except:
                  image_url = ""

              wrp_content_element = each_item_element.find('div', class_='wrp-content')

              title = wrp_content_element.find('div', class_='title').text

              wrp_price_element = wrp_content_element.find('div', class_='wrp-price')

              # cek apakah ada discount atau tidak
              try:
                  if wrp_price_element.find('span', class_='strikeout disc-price'):
                      item_price = wrp_price_element.find('span', class_='discount').text
                      item_price = re.sub(r"<span>.*?</span>", "", item_price)
                  else:
                      price_element = wrp_price_element.find('div', class_='price')
                      item_price = price_element.find('span', class_='normal price-value').text
              except:
                  item_price = 'tidak ada'
                  print(f'gagal mendapatkan harga untuk item: {title}')

              products.append({
                  'title': title.strip(),
                  'normal_price': item_price.strip(),
                  'label': label,
                  'image_url': image_url,
              })

          print(f"berhasil melakukan percobaan ke-{page_index}")
      except Exception as e:
          print("terjadi error: ", e)

      driver.quit()

  return products

Function to save data in the form of a csv file

In [ ]:
def save_to_csv(data, filename):
    folder_name = 'data'

    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

    filepath = os.path.join(folder_name, f"{filename}.csv")

    with open(filepath, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"berhasil menyimpan ke file {filepath}")

List of old categories that are no longer relevant

In [ ]:
# categories = [
    # {
    #     'key': 'Kesehatan',
    #     'categories': 'kesehatan',
    #     'max_page': 6,
    #     'label': 'kesehatan',
    # },
    # {
    #     'key': 'Segar',
    #     'categories': 'ayam',
    #     'max_page': 2,
    #     'label': 'produk segar',
    # },
    # {
    #     'key': 'Makanan',
    #     'categories': 'Makanan',
    #     'max_page': 18,
    #     'label': 'makanan',
    # },
    # {
    #     'key': 'minuman',
    #     'categories': 'beverage',
    #     'max_page': 9,
    #     'label': 'minuman',
    # },
    # {
    #     'key': 'kecantikan',
    #     'categories': 'personal-care',
    #     'max_page': 4,
    #     'label': 'kecantikan',
    # },
# ]

In [ ]:
# for category in categories:
#     data = scrapper(
#         key=category['key'],
#         categories=category['categories'],
#         label=category['label'],
#         max_page=category['max_page'],
#     )

#     save_to_csv(data, filename=category['label'])

memulai scrapping untuk label: produk segar

percobaan ke-1: https://www.klikindomaret.com/search/?key=Segar&categories=ayam&productbrandid=&sortcol=&pagesize=54&page=1&startprice=&endprice=&attributes=&ShowItem=
berhasil melakukan percobaan ke-1
percobaan ke-2: https://www.klikindomaret.com/search/?key=Segar&categories=ayam&productbrandid=&sortcol=&pagesize=54&page=2&startprice=&endprice=&attributes=&ShowItem=
berhasil melakukan percobaan ke-2


IndexError: list index out of range

Scrapping process uses a loop based on some defined categories

In [ ]:
# new scrapping method
scrapper_data = [
    {
        'category1': 'makanan-beku',
        'category2': 'Makanan-Beku',
        'max_page': 2,
        'label': 'product-segar-makanan-beku',
    },
    {
        'category1': 'buah-dan-dessert',
        'category2': 'buah-dan-dessert',
        'max_page': 3,
        'label': 'product-segar-buah-dan-dessert',
    },
    {
        'category1': 'olahan-susu-telur',
        'category2': 'olahan-susu-telur',
        'max_page': 2,
        'label': 'product-segar-olahan-susu-telur',
    },
    {
        'category1': 'es-krim',
        'category2': 'es-krim',
        'max_page': 3,
        'label': 'product-segar-es-krim',
    },
    {
        'category1': 'aneka-roti',
        'category2': 'aneka-roti',
        'max_page': 5,
        'label': 'product-segar-aneka-roti',
    },
    {
        'category1': 'sayur',
        'category2': 'sayur',
        'max_page': 1,
        'label': 'product-segar-sayur',
    },
    {
        'category1': 'makanan-siap-saji',
        'category2': 'makanan-siap-saji',
        'max_page': 1,
        'label': 'product-segar-makanan-siap-saji',
    },
    {
        'category1': 'dagingayam--seafood',
        'category2': 'dagingayam--seafood',
        'max_page': 2,
        'label': 'product-segar-dagingayam--seafood',
    },

    # makanan
    {
        'category1': 'sarapan',
        'category2': 'sarapan',
        'max_page': 3,
        'label': 'makanan-sarapan',
    },
    {
        'category1': 'makanan-kaleng',
        'category2': 'makanan-kaleng',
        'max_page': 1,
        'label': 'makanan-makanan-kaleng',
    },
    {
        'category1': 'bahan-makanan',
        'category2': 'bahan-makanan',
        'max_page': 9,
        'label': 'makanan-bahan-makanan',
    },
    {
        'category1': 'cokelat--permen',
        'category2': 'cokelat--permen',
        'max_page': 5,
        'label': 'makanan-cokelat--permen',
    },
    {
        'category1': 'makanan-instan',
        'category2': 'Makanan-Instan',
        'max_page': 4,
        'label': 'makanan-makanan-instan',
    },
    {
        'category1': 'cemilan--biskuit',
        'category2': 'Cemilan--Biskuit',
        'max_page': 12,
        'label': 'makanan-cemilan--biskuit',
    },
    {
        'category1': 'healthy-food',
        'category2': 'healthy-food',
        'max_page': 1,
        'label': 'makanan-healthy-food',
    },
    {
        'category1': 'korean-food',
        'category2': 'korean-food',
        'max_page': 1,
        'label': 'makanan-korean-food',
    },
    {
        'category1': 'oriental-food',
        'category2': 'oriental-food',
        'max_page': 1,
        'label': 'makanan-oriental-food',
    },
    {
        'category1': 'western-food',
        'category2': 'western-food',
        'max_page': 1,
        'label': 'makanan-western-food',
    },
    {
        'category1': 'bahan-kue',
        'category2': 'bahan-kue',
        'max_page': 4,
        'label': 'makanan-bahan-kue',
    },

    # minuman
    {
        'category1': 'kental-manis',
        'category2': 'kental-manis',
        'max_page': 1,
        'label': 'minuman-kental-manis',
    },
    {
        'category1': 'susu',
        'category2': 'susu',
        'max_page': 6,
        'label': 'minuman-susu',
    },
    {
        'category1': 'minuman-ringan',
        'category2': 'minuman-ringan',
        'max_page': 3,
        'label': 'minuman-minuman-ringan',
    },
    {
        'category1': 'jus',
        'category2': 'jus',
        'max_page': 1,
        'label': 'minuman-jus',
    },
    {
        'category1': 'kopi',
        'category2': 'kopi',
        'max_page': 3,
        'label': 'minuman-kopi',
    },
    {
        'category1': 'teh',
        'category2': 'teh',
        'max_page': 2,
        'label': 'minuman-teh',
    },
    {
        'category1': 'air-mineral',
        'category2': 'air-mineral',
        'max_page': 1,
        'label': 'minuman-air-mineral',
    },
    {
        'category1': 'sirup',
        'category2': 'sirup',
        'max_page': 1,
        'label': 'minuman-sirup',
    },
    {
        'category1': 'coklat-bubuk-kremer',
        'category2': 'coklat-bubuk-kremer',
        'max_page': 1,
        'label': 'minuman-coklat-bubuk-kremer',
    },
    {
        'category1': 'minuman-tradisional',
        'category2': 'minuman-tradisional',
        'max_page': 1,
        'label': 'minuman-minuman-tradisional',
    },

    # kecantikan - personal care
    {
        'category1': 'cosmetic',
        'category2': 'cosmetic',
        'max_page': 14,
        'label': 'kecantikan-cosmetic',
    },
    {
        'category1': 'perawatan-diri',
        'category2': 'Perawatan-Diri',
        'max_page': 5,
        'label': 'kecantikan-perawatan-diri',
    },
    {
        'category1': 'body-care',
        'category2': 'body-care',
        'max_page': 8,
        'label': 'kecantikan-body-care',
    },
    {
        'category1': 'hair-care',
        'category2': 'hair-care',
        'max_page': 5,
        'label': 'kecantikan-hair-care',
    },
    {
        'category1': 'mens-care',
        'category2': 'mens-care',
        'max_page': 3,
        'label': 'kecantikan-mens-care',
    },

    # kesehatan
    {
        'category1': 'masker',
        'category2': 'masker',
        'max_page': 1,
        'label': 'kesehatan-masker',
    },
    {
        'category1': 'antiseptik--sanitasi',
        'category2': 'antiseptik--sanitasi',
        'max_page': 1,
        'label': 'kesehatan-antiseptik--sanitasi',
    },
    {
        'category1': 'medicine--food-supplement',
        'category2': 'medicine--food-supplement',
        'max_page': 1,
        'label': 'kesehatan-medicine--food-supplement',
    },
    {
        'category1': 'obat--luar',
        'category2': 'obat--luar',
        'max_page': 2,
        'label': 'kesehatan-obat--luar',
    },
    {
        'category1': 'internal-medicine',
        'category2': 'internal-medicine',
        'max_page': 1,
        'label': 'kesehatan-internal-medicine',
    },
    {
        'category1': 'alat-kesehatan',
        'category2': 'alat-kesehatan',
        'max_page': 1,
        'label': 'kesehatan-alat-kesehatan',
    },

    # home & living
    {
        'category1': 'jas-hujan',
        'category2': 'jas-hujan',
        'max_page': 1,
        'label': 'home-living-jas-hujan',
    },
    {
        'category1': 'produk-tisu',
        'category2': 'produk-tisu',
        'max_page': 1,
        'label': 'home-living-produk-tisu',
    },
    {
        'category1': 'deterjen-dan-pembersih',
        'category2': 'deterjen-dan-pembersih',
        'max_page': 5,
        'label': 'home-living-deterjen-dan-pembersih',
    },
    {
        'category1': 'pembasmi-kuman-dan-pewangi',
        'category2': 'pembasmi-kuman-dan-pewangi',
        'max_page': 4,
        'label': 'home-living-pembasmi-kuman-dan-pewangi',
    },
    {
        'category1': 'household-non-electric--hobby-product',
        'category2': 'household-non-electric--hobby-product',
        'max_page': 3,
        'label': 'home-living-household-non-electric--hobby-product',
    },
    {
        'category1': 'gas-elpiji',
        'category2': 'gas-elpiji',
        'max_page': 1,
        'label': 'home-living-gas-elpiji',
    },
    {
        'category1': 'handuk',
        'category2': 'handuk',
        'max_page': 1,
        'label': 'home-living-handuk',
    },
    {
        'category1': 'mekanik-elektrik',
        'category2': 'mekanik-elektrik',
        'max_page': 2,
        'label': 'home-living-mekanik-elektrik',
    },
    {
        'category1': 'stationery--special-items',
        'category2': 'stationery--special-items',
        'max_page': 3,
        'label': 'home-living-stationery--special-items',
    },
    {
        'category1': 'pembasmi-kuman-dan-pewangi',
        'category2': 'pembasmi-kuman-dan-pewangi',
        'max_page': 4,
        'label': 'home-living-pembasmi-kuman-dan-pewangi',
    },
    {
        'category1': 'perlengkapan--makanan-hewan',
        'category2': 'perlengkapan--makanan-hewan',
        'max_page': 2,
        'label': 'home-living-perlengkapan--makanan-hewan',
    },
]

for item in scrapper_data:
    result = scrapper_v2(
        category1=item['category1'],
        category2=item['category2'],
        max_page=item['max_page'],
        label=item['label'],
    )

    save_to_csv(result, filename=item['label'])

memulai scrapping untuk label: makanan-healthy-food

percobaan ke-1: https://www.klikindomaret.com/category/healthy-food?categories=healthy-food&pagesize=54&page=1
berhasil melakukan percobaan ke-1
berhasil menyimpan ke file data/makanan-healthy-food.csv
memulai scrapping untuk label: makanan-korean-food

percobaan ke-1: https://www.klikindomaret.com/category/korean-food?categories=korean-food&pagesize=54&page=1
berhasil melakukan percobaan ke-1
berhasil menyimpan ke file data/makanan-korean-food.csv
memulai scrapping untuk label: makanan-oriental-food

percobaan ke-1: https://www.klikindomaret.com/category/oriental-food?categories=oriental-food&pagesize=54&page=1
berhasil melakukan percobaan ke-1
berhasil menyimpan ke file data/makanan-oriental-food.csv
memulai scrapping untuk label: makanan-western-food

percobaan ke-1: https://www.klikindomaret.com/category/western-food?categories=western-food&pagesize=54&page=1
berhasil melakukan percobaan ke-1
berhasil menyimpan ke file data/makan

Create a zip file so that the dataset can be downloaded to a local device

In [ ]:
!zip -r "/content/data.zip" "/content/data"

  adding: content/data/ (stored 0%)
  adding: content/data/product-segar-makanan-beku.csv (deflated 85%)
  adding: content/data/minuman-sirup.csv (deflated 69%)
  adding: content/data/.ipynb_checkpoints/ (stored 0%)
  adding: content/data/kesehatan-internal-medicine.csv (deflated 80%)
  adding: content/data/home-living-deterjen-dan-pembersih.csv (deflated 86%)
  adding: content/data/makanan-western-food.csv (deflated 78%)
  adding: content/data/makanan-bahan-makanan.csv (deflated 85%)
  adding: content/data/minuman-jus.csv (deflated 81%)
  adding: content/data/product-segar-aneka-roti.csv (deflated 86%)
  adding: content/data/product-segar-es-krim.csv (deflated 85%)
  adding: content/data/minuman-susu.csv (deflated 86%)
  adding: content/data/minuman-minuman-ringan.csv (deflated 84%)
  adding: content/data/makanan-makanan-kaleng.csv (deflated 82%)
  adding: content/data/product-segar-sayur.csv (deflated 81%)
  adding: content/data/minuman-kental-manis.csv (deflated 83%)
  adding: conte

In [ ]:
# unzip data dari lokal file
!unzip "/content/data.zip" -d 'imported_data'

import shutil
import os

source_dir = os.path.join('imported_data', 'content', 'data')
destination_dir = os.path.join('data')

for filename in os.listdir(source_dir):
  source_path = os.path.join(source_dir, filename)
  destination_path = os.path.join(destination_dir, filename)

  if os.path.isfile(source_path):
    shutil.move(source_path, destination_path)

shutil.rmtree(os.path.join('imported_data'))

Archive:  /content/data.zip
replace imported_data/content/data/product-segar-makanan-beku.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: imported_data/content/data/product-segar-makanan-beku.csv  
  inflating: imported_data/content/data/minuman-sirup.csv  
  inflating: imported_data/content/data/kesehatan-internal-medicine.csv  
  inflating: imported_data/content/data/home-living-deterjen-dan-pembersih.csv  
  inflating: imported_data/content/data/makanan-western-food.csv  
  inflating: imported_data/content/data/makanan-bahan-makanan.csv  
  inflating: imported_data/content/data/minuman-jus.csv  
  inflating: imported_data/content/data/product-segar-aneka-roti.csv  
  inflating: imported_data/content/data/product-segar-es-krim.csv  
  inflating: imported_data/content/data/minuman-susu.csv  
  inflating: imported_data/content/data/minuman-minuman-ringan.csv  
  inflating: imported_data/content/data/makanan-makanan-kaleng.csv  
  inflating: imported_data/content/data/product-

Function to merge multiple csv files into one

In [ ]:
def merge_csv_files(input_filepaths, output_filename):
    output_filepath = os.path.join('merged_data', output_filename)

    if not os.path.exists(os.path.join('merged_data')):
        os.mkdir(os.path.join('merged_data'))

    with open(output_filepath, mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        header_written = False

        for filepath in input_filepaths:
            with open(filepath, mode='r') as file_item:
                reader = csv.reader(file_item)
                header = next(reader)

                if not header_written:
                    writer.writerow(header)
                    header_written = True

                for row in reader:
                    writer.writerow(row)

    return output_filepath

In [ ]:
product_labels = [
    'home-living',
    'kecantikan',
    'kesehatan',
    'makanan',
    'minuman',
    'product-segar',
]

for product_label in product_labels:
    input_filepaths = [
        os.path.join('data', filename) for filename in os.listdir('data')
        if filename.startswith(product_label) and
           filename.endswith('.csv')
    ]

    merge_csv_files(input_filepaths, f'{product_label}.csv')

In [ ]:
# merge seluruh product per kategori menjadi satu file csv

if not os.path.exists(os.path.join('merged_data_2')):
  os.mkdir(os.path.join('merged_data_2'))

with open(os.path.join('merged_data_2', 'merged_data.csv'), mode='w', newline='') as output_file:
  writer = csv.writer(output_file)
  writer.writerow(['product_name', 'label', 'price'])

  for filename in os.listdir('merged_data'):
    if filename.endswith('.csv'):
      label = filename.split('.')[0]
      filepath = os.path.join('merged_data', filename)

      with open(filepath, mode='r') as infile:
        reader = csv.reader(infile)

        for row in reader:
          if row:
            writer.writerow([row[0], label, row[1]])

The process of randomizing each row in the csv file so that the data distribution becomes more evenly distributed during the training and validation process.

In [ ]:
import pandas as pd

def shuffle_csv(csv_filepath, output_filepath):
    data_frame = pd.read_csv(csv_filepath)
    data_frame = data_frame.sample(frac=1).reset_index(drop=True)
    data_frame.to_csv(output_filepath, index=False)

    return output_filepath

In [ ]:
# acak csv yang sudah digabungkan
shuffle_csv(
    csv_filepath=os.path.join('merged_data_2', 'merged_data.csv'),
    output_filepath=os.path.join('merged_data_2', 'shuffled_data.csv'),
)

'merged_data_2/shuffled_data.csv'